# 训练和测试集创建

该项目的第一步包括训练和测试集的创建。

## 特征提取

数据集是使用 Features 类创建的。
每个音频文件都加载到内存中，并提取以下特征：
- mfcc 梅尔频率倒谱系数
- chroma 色度
- rms 均方根

然后使用以下函数减少每个特征数组：
- min 最小值
- max 最大限度
- median 中位数
- mean 均值

将结果连接起来，从每个音频中提取总共 132 个特征。

## 结构

数据集按以下结构组织

$$\mathit{class}, \; \mathit{feature}_1, \; \dots, \; \mathit{feature}_n$$

## 缩放

将标准缩放器应用于训练集并将其保存到磁盘。
当处理测试折叠时，相同的缩放器会应用于数据。

## Dask 加速

为了加快计算速度，使用了 Dask。

总共有 4 个 worker 并行工作，以更有效地提取特征，将单个文件夹的时间从大约 70 秒减少到不到 30 秒。

## 训练数据集

第一步是获取训练数据集，考虑的文件夹是**前四个文件夹和第六个文件夹**。获取的数据集中的样本总数为 4499。


## 环境设置

In [ ]:
!pip install wanglaoshi

In [ ]:
from wanglaoshi import JupyterEnv as JE
JE.jupyter_kernel_list()

In [ ]:
JE.install_kernel()

> 创建完之后，刷新环境，选择对应的库，然后安装依赖库。

In [ ]:
!pip install -r /mnt/workspace/urban-sound-classification/src/requirements.txt

In [ ]:
!pip install dask

In [ ]:
!pip install "dask[distributed]" --upgrade

## 拉取数据

在命令行工具中依次执行


![0oKZjO](https://upiclw.oss-cn-beijing.aliyuncs.com/uPic/0oKZjO.png)

```shell
mkdir /mnt/workspace/urban-sound-classification/data/raw/
```

```shell
git clone https://github.com/WangLaoShi/UrbanSound8K.git /mnt/workspace/urban-sound-classification/data/raw/
```

```shell
ls /mnt/workspace/urban-sound-classification/data/raw
```

```shell
mv /mnt/workspace/urban-sound-classification/data/raw/UrbanSound8K/metadata/ /mnt/workspace/urban-sound-classification/data/raw/
```

```shell
mv /mnt/workspace/urban-sound-classification/data/raw/UrbanSound8K/audio/ /mnt/workspace/urban-sound-classification/data/raw/
```



In [30]:
import librosa
print(librosa.__version__)

0.10.1


In [33]:
import sys
sys.path.append("..")
from src.data import Features
import pandas as pd
import numpy as np

## 未缩放的训练集
以下单元格提取未缩放的训练集。

In [ ]:
f = Features(save_path="../data/processed/initial",
             save_name="train_unscaled",
             folds=[1,2,3,4,6])

training_dataframe = f.get_dataframe()
f.save_dataframe(training_dataframe)

In [ ]:
training_dataframe

## 缩放数据集
将标准缩放器应用于数据集并保存以供稍后在测试集上进行缩放。

In [ ]:
scaled_df = f.scale_dataframe(training_dataframe, 
                              save_scaler=True)
scaled_df

In [ ]:
f.save_dataframe(scaled_df, save_name="train_scaled")

## 测试数据集

得到训练集之后，从其他文件夹中获取多个测试集。

每一个都以缩放和未缩放的形式保存，以测试缩放改进。

In [ ]:
for fold in [5, 7, 8, 9, 10]:
    print(f"处理文件夹 {fold}")
    
    f = Features(save_path="../data/processed/initial",
                 save_name=f"test_{fold}_unscaled",
                 folds=[fold])
    
    df = f.get_dataframe()
    f.save_dataframe(df)
    
    scaled = f.apply_scaling(df, "../models/scalers/scaler_training.pkl")
    f.save_dataframe(scaled, save_name=f"test_{fold}_scaled")